# Advent of Code 2017, Dyalog APL edition

To see a correct render of this notebook, check it out on [nbviewer](https://nbviewer.jupyter.org/github/xpqz/AoCDyalog/blob/master/Advent%20of%20Code%202017%20Dyalog%20APL.ipynb).

Annotated solutions in Dyalog APL.

Note that part of the charm of AoC is that every user (or at least groups of users) gets their own unique data set. Some of the solutions below exploit quirks in my particular data set, and so may conceivably not work for the general case.

In [38]:
⎕IO←1
]box on -style=max -trains=tree -fns=on
]rows on

┌→─────────────────────────────────────┐
│Was ON -style=max -trains=tree -fns=on│
└──────────────────────────────────────┘

┌→──────┐
│Was OFF│
└───────┘

### Day 1: Inverse Captcha
https://adventofcode.com/2017/day/1

In [5]:
DAY1←⊃⊃⎕NGET'data/2017/01.txt'1

Tack on the first item to the end, as per the problem statement.

In [55]:
DATA←DAY1,1⌷DAY1

We use a length-2 windowed reduction using equality to find all location where item n is equal to item n+1 as a binary map. We convert this to position (`⍸`) and pick the corresponding items. As we're still dealing with characters, we need to convert to integers (`⍎¨`) and then just sum them up.

In [56]:
+/⍎¨DATA[⍸2=/DATA] ⍝ Part 1: 1341

1341

For part 2, we create a matrix where column 1 is the input and column 2 is the data rotated by half its length. Then compare the elements of each row to produce the binary map, and the rest is as per part 1.

In [17]:
+/⍎¨DAY1[⍸=/⍉↑(DAY1)(DAY1⊖⍨2÷⍨≢DAY1)] ⍝ Part 2: 1348

1348

### Day 2: Corruption Checksum
https://adventofcode.com/2017/day/2

In [52]:
DAY2←16 16⍴⍎¨'\d+'⎕S'&'⊢⊃⎕NGET'data/2017/02.txt'1

Part 1 -- sum max-min for every row

In [53]:
+/(⌈/-⌊/)DAY2 ⍝ Part 1: 39126

39126

In part 2 we're looking for the single pair of items per row that divides cleanly. For each row we generate all combinations, and try both ways of modular division, looking for a zero (not two) using 

    {≠/0=⍺(|,|⍨)⍵}
    
Finally, return the larger÷smaller and sum up everything.

In [79]:
Day2←{target←1↑⍸{≠/0=⍺(|,|⍨)⍵}/¨cmb←,⍵∘.,⍵⋄÷/pair[⍒pair←target⊃cmb]}

In [80]:
+/Day2¨↓DAY2 ⍝ Part 2: 258

258

### Day 3: Spiral Memory
https://adventofcode.com/2017/day/3

This [spiral](http://www.mathrecreation.com/2011/06/sequences-on-spiral.html) has a number of interesting properties. See http://oeis.org/A080335

We can exploit that we have perfect squares on the SE diagonal, which allows us to identify the size of the square which has the target number along one of its edges. We can then identify which edge, and count backwards from the next corner of the square (anti-clockwise).

In [83]:
⎕IO←0
DAY3←325489

In [82]:
Square←{r←⍵*0.5⋄⍵=2*⍨⌊r:r⋄0≠2|⌈r:⌈r⋄1+⌈r} ⍝ Smallest odd number < the square root, unless perfect square

In [110]:
]dinput
Day3p1←{
    val←⍵
    width←Square ⍵                    ⍝ The width of the square in which the value is found
    edge←⌊((width*2)-val)÷(width-1)   ⍝ The side of the square: S E N W ← 0 1 2 3
    pos←edge⊃,size,-size∘.,-size,size←⌊width÷2  ⍝ Grid coords of corner
    _←(edge⊃,¯1 0∘.,0 ¯1)∘{pos+←⍺⋄⍵-1}⍣{⍺=val} (width*2)-edge×width-1  ⍝ Sequence at the nearest corner, acw
    +/|¨pos                           ⍝ Manhattan distance to centre
}

In [111]:
Day3p1 DAY3 ⍝ 552

552

For part 2, we work our way outwards following the same spiral pattern, but this time the value of each point is the sum of the values of its 8-connected neighbours _at the time of visit_. The APL solution isn't pretty.

In [119]:
]dinput
Coords←{
    ⍝ Generate all coordinate pairs for the circumference of a square ⍵
    pos←(⌊⍵÷2),(-⌊⍵÷2)+1
    
    E←(⊂pos)+0,¨⍳⍵-1
    N←(¯1↑E)+(-⍳⍵),¨0
    W←(¯1↑N)+0,¨-⍳⍵
    S←(¯1↑W)+(⍳⍵),¨0
    ∪E, N, W, S
}

In [116]:
Neighbours←{(⊂⍵)+(0 1)(1 1)(1 0)(1 ¯1)(0 ¯1)(¯1 ¯1)(¯1 0)(¯1 1)} ⍝ 8-neighbours

In [117]:
]dinput
Day3p2←{
    target←⍵
    seen←,⊂0 0 ⋄ vals←,1
    { ⍝ Move outwards one square at a time (i.e. odd numbers)
        found←{ ⍝ Walk circumference, adding up values from visible neighbours
            0=≢⍵:0
            pos←⊃1↑⍵
            old←seen⍳Neighbours pos
            newVal←+/vals[((≢seen)>old)/old]
            newVal>target:newVal
            vals,←newVal
            seen,←⊂pos
            ∇1↓⍵
        } Coords ⍵
        found>0:found
        ∇⍵+2
    } 3
}

In [120]:
Day3p2 325489 ⍝ 330785

330785

### Day 4: High-Entropy Passphrases
https://adventofcode.com/2017/day/4

In [130]:
⎕IO←1
DAY4←' '(≠⊆⊢)¨⊃⎕NGET'data/2017/04.txt'1

Find phrases which don't match themselves with dupes removed.

In [142]:
+/(∪≡⊢)¨DAY4 ⍝ Part 1: 325

325

Part 2: apply the same process, but sort all words first.

In [151]:
+/(∪≡⊢)¨{⍵[⍋⍵]}¨¨DAY4 ⍝ Part 2: 119

119

### Day 5: A Maze of Twisty Trampolines, All Alike
https://adventofcode.com/2017/day/5

In [155]:
⎕IO←1
DAY5←⍎¨⊃⎕NGET'data/2017/05.txt'1

In [169]:
]dinput
Day5←{
    instr←⍵
    modifier←⍺⍺
    1 {
        ip←1↑⍵
        jmp←ip⌷instr
        (ip+jmp)>≢instr:⍺
        instr[ip]+←modifier jmp
        (⍺+1)∇ip+jmp
    } 1
}

In [167]:
{1} Day5 DAY5 ⍝ Part 1: 372671

372671

In [168]:
{⍵≥3:¯1⋄1} Day5 DAY5 ⍝ Part 2: 25608480 (takes a minute or so)

25608480

### Day 6: Memory Reallocation
https://adventofcode.com/2017/day/6

Part 1 and 2 differs only in the end condition and the start value. 

In [219]:
⎕IO←0
DAY6←2 8 8 5 4 2 3 1 5 5 1 2 15 13 5 14

In [229]:
]dinput
Redist←{
    (0@m⊢⍵) {                        ⍝ Zero out the current bin
        0=⊃⍵:⍺                       ⍝ Return end state
        (1∘+@n⊢⍺)∇(⊃⍵-1),n←16|1+⊢/⍵  ⍝ Move one index on, mod size and add 1. Decrease value to distribute.
    } ⍵[m],m←(⊢⍳⌈/)⍵                 ⍝ Value and index of the largest element
}

In [230]:
Day6p1←{seen←⊂⍵⋄e←Redist⍣{seen∊⍨⊂⍺:1⋄0⊣seen,←⊂⍺} ⍵⋄(⊂e),≢seen} ⍝ Stop if we've seen a state before.

In [231]:
(elem seen)←Day6p1 DAY6
seen ⍝ 3156

3156

In [232]:
Day6p2←{count←0⋄target←⍵⋄_←Redist⍣{count+←1⋄⍺≡target} ⍵⋄count} ⍝ Stop when we revisit end state of part 1.

In [233]:
Day6p2 elem ⍝ Part 2: 1610

1610

### Day 7: Recursive Circus
https://adventofcode.com/2017/day/7

In [238]:
⎕IO←1
'segs'⎕CY'dfns'
DAY7←⊃⎕NGET'data/2017/07.txt'1

We need to create a tree structure from the data. We'll create three vectors, the first being a list of the nodes themselves, the second holding their corresponding weights, and the third a nested vector containing any child-nodes. The 'dfns' workspace contains a handy function 'segs' that can slice up strings on a set of separators.

In [243]:
]dinput
Parse←{
    ⍺←⍬ ⍬ ⍬
    0=≢⍵:⍺
    items←'() ,->' segs ⊃1↑⍵
    (⍺,¨(⊂1⊃items)(⍎2⊃items)(⊂2↓items))∇1↓⍵
}

In [245]:
(nodes weights children)←Parse DAY7
⊃nodes~{⊃,/,⊆¨⍵}⍣≡children ⍝ Part 1: xegshds

┌→──────┐
│xegshds│
└───────┘

For part 2 we are to identify a single weight tweak to make the tree balanced in the sense that every child tree of any given node has equal weights. The first part of that is to be able to calculate the weight of the tree from a specific start node:

In [246]:
]dinput
SubTreeWeight←{              ⍝ (nodes weights children) SubTreeWeight 'node'
    idx←(1⊃⍺)⍳⊂⍵             ⍝ Find the node's index
    0=≢idx⊃3⊃⍺:idx⊃2⊃⍺       ⍝ If node has no children -- return the weight
    (idx⊃2⊃⍺)++/⍺∘∇¨idx⊃3⊃⍺  ⍝ Add own weight to sum of child tree weights recursively
}

We can now drill down from the root, looking for the first node that is balanced. The tweak needs to happen at the parent level of this node. We descend recursively following the odd one out of the child nodes until all child nodes have the same weight.

In [281]:
]dinput
FindAnomaly←{ ⍝ Find first key where its child trees all have the same weight
    idx←(1⊃⍺)⍳⊂⍵
    ch←idx⊃3⊃⍺
    0=≢ch:⍬
    chw←⍺∘SubTreeWeight¨ch
    (1≥≢∘∪)chw:⍵ (idx)      ⍝ Child tree weights all equal - we're done
    ⍺∇⊃ch[∊{∩/⍵}⌸chw]       ⍝ Intersect-reduce over unique indexes only returns non-empty for singles.
}

In [282]:
⊢ANOMALY←(nodes weights children) FindAnomaly 'xegshds'

┌→──────────────┐
│ ┌→──────┐ │
│ │fabacam│ 982 │
│ └───────┘ │
└∊──────────────┘

The size of the tweak we need to make to the anomaly is the difference of its parent's child weights.

In [283]:
PARENT←⍸{ANOMALY[1]∊⍵}¨children

In [279]:
TWEAK←|-/∪(nodes weights children)∘SubTreeWeight¨⊃children[PARENT]

In [280]:
weights[2⊃ANOMALY]-TWEAK ⍝ Part 2: 299

299

### Day 8: I Heard You Like Registers
https://adventofcode.com/2017/day/8

In [340]:
⎕IO←1
'segs'⎕CY'dfns'
DAY8←⊃⎕NGET'data/2017/08.txt'1

In [341]:
DATA←' ' segs¨DAY8

In [361]:
]dinput
Day8←{
    regs←∪{⊃,/,⊆¨⍵}⍣≡(↑⍵)[;1 5]
    R←regs∘⍳                       ⍝ By binding an operator, the static 'regs' vector is hashed
    vals←0⍴⍨≢regs
    Reg←{vals[R ⊂,⍵]}
    Set←{vals[R ⊂,⍺]←⍵⋄⍬}
    inc←{⍺ Set (Reg ⍺)+⍎⍵}
    dec←{⍺ Set (Reg ⍺)-⍎⍵}
    0 {
        0=≢⍵:(⌈/vals) ⍺
        max←⌈/vals,⍺
        instr←1⊃⍵
        rv←Reg 5⊃instr
        f←⍎2⊃instr
        ((,'>')≡6⊃instr)∧rv>⍎7⊃instr:max∇1↓⍵⊣(1⊃instr) f 3⊃instr
        ((,'<')≡6⊃instr)∧rv<⍎7⊃instr:max∇1↓⍵⊣(1⊃instr) f 3⊃instr
        ('=='≡6⊃instr)∧rv=⍎7⊃instr:max∇1↓⍵⊣(1⊃instr) f 3⊃instr
        ('>='≡6⊃instr)∧rv≥⍎7⊃instr:max∇1↓⍵⊣(1⊃instr) f 3⊃instr
        ('<='≡6⊃instr)∧rv≤⍎7⊃instr:max∇1↓⍵⊣(1⊃instr) f 3⊃instr
        ('!='≡6⊃instr)∧rv≠⍎7⊃instr:max∇1↓⍵⊣(1⊃instr) f 3⊃instr
        max∇1↓⍵
    } ⍵
}

In [362]:
Day8 DATA ⍝ Part 1: 3745 Part2: 4644

┌→────────┐
│3745 4644│
└~────────┘

### Day 9: Stream Processing
https://adventofcode.com/2017/day/9

In [366]:
⎕IO←1
DAY9←⊃⊃⎕NGET'data/2017/09.txt'1

In [376]:
]dinput
Day9←{
    pos←0
    str←⍵
    Yield←{pos+←1⋄pos⊃str}
    Skip←{⍺←0⋄c←Yield⍬⋄c='>':⍺⋄c='!':⍺∇⍬⊣Yield⍬⋄(⍺+1)∇⍬}
    (0 0 0) {
        3::1↓⍺  ⍝ Catch index error at end of input
        (depth total skipped)←⍺
        c←Yield⍬
        c='{':((1+depth),total,skipped)∇⍬
        c='}':((¯1+depth),(total+depth),skipped)∇⍬
        c='<':(depth,total,skipped+Skip⍬)∇⍬
        c='!':⍺∇⍬⊣Yield⍬
        ⍺∇⍬
    }⍬
}

In [377]:
Day9 DAY9 ⍝ Part 1: 10050 Part 2: 4482

┌→─────────┐
│10050 4482│
└~─────────┘

### Day 10: Knot Hash
https://adventofcode.com/2017/day/10

In [23]:
⎕IO←0
'iotag'⎕CY'dfns'
DAY10←147 37 249 1 31 2 226 0 161 71 254 243 183 255 30 70

In [24]:
]dinput
Rot←{
    (skip pos len)←⍵
    0=len:(skip+1) (256|pos+len+skip) ⍺
    (skip+1) (256|pos+len+skip) (⊖@(256|pos iotag pos+len-1)⊢⍺)
}

In [25]:
Round←{0=≢⍵:⍺⋄((2⊃⍺) Rot (0⊃⍺) (1⊃⍺) (0⊃⍵))∇1↓⍵}

Part 1 is a single round of the knot hash algorithm, with the answer sought is the product of the two first numbers.

In [436]:
×/2↑2⊃(0 0,⊂⍳256)Round DAY10 ⍝ Part 1: 37230

37230

For part 2, we apply the knot hash round 64 times. The lengths are now a byte array derived from a string representation of the data (including commas!) and 5 additional bytes added at the end.

After the 64 rounds, we partition the result into blocks of 16:

    ↓16 16 ⍴ sparse
    
and XOR-reduce each block:

    {2⊥⊃≠/(8⍴2)∘⊤¨⍵}¨
    
and finally convert to hexadecimal:

    ↓⍉{(⎕D,⎕A)[16⊥⍣¯1⊢⍵]}


In [26]:
]dinput
Knot←{
    lengths←(⎕UCS¨' '⎕R','⍕⍵),17 31 73 47 23
    sparse←(0 0,⊂⍳256) {
        0=⍵:2⊃⍺
        (⍺ Round lengths)∇⍵-1
    } 64
    ↓⍉{(⎕D,⎕A)[16⊥⍣¯1⊢⍵]} {2⊥⊃≠/(8⍴2)∘⊤¨⍵}¨↓16 16 ⍴ sparse
}

In [27]:
Knot DAY10 ⍝ 70b856a24d586194331398c7fcfa0aaf

┌──┬──┬──┬──┬──┬──┬──┬──┬──┬──┬──┬──┬──┬──┬──┬──┐
│70│B8│56│A2│4D│58│61│94│33│13│98│C7│FC│FA│0A│AF│
└──┴──┴──┴──┴──┴──┴──┴──┴──┴──┴──┴──┴──┴──┴──┴──┘

### Day 11: Hex Ed
https://adventofcode.com/2017/day/11

Cube coordinates for hex grids; see https://www.redblobgames.com/grids/hexagons/#coordinates

Map N-S axis to y, NE-SW to z and NW-SE to x

Manhattan distance on the hex grid is half that on the cube grid.

We start with splitting the input on comma, and then converting strings to the index of their first occurrance. Note that this ordering needs to be mirrored in the DELTA variable of offsets -- it's not general for all inputs.

In [474]:
⎕IO←1
DAY11←(∪⍳⊢)','(≠⊆⊢)⊃⊃⎕NGET'data/2017/11.txt'1 

In [478]:
HexMHD←{⌊2÷⍨+/|⍵} ⍝ Manhattan distance on a hex grid

In [476]:
DELTA←(¯1 0 1)(0 ¯1 1)(1 ¯1 0)(¯1 1 0)(1 0 ¯1)(0 1 ¯1) ⍝ SW S SE NW NE N -- order of index of first occurrance

All we need to do is sum it up - but as Dyalog's reduce goes R-L we need to reverse.

In [479]:
HexMHD ⊃+/⊖DELTA[DAY11] ⍝ Part 1: 643

643

For part 2, find the max MHD attained whilst walking the path. Dyalog's scan reduction operator goes left to right and maintains the running total.

In [491]:
⌈/HexMHD¨+\DELTA[DAY11] ⍝ Part 2: 1471

1471

### Day 12: Digital Plumber
https://adventofcode.com/2017/day/12

In [503]:
⎕IO←0
'segs'⎕CY'dfns'
DAY12←1↓¨⍎¨¨' <->,'∘segs¨⊃⎕NGET'data/2017/12.txt'1 

Discover all nodes reachable from the root by means of a classic breadth-first search.

In [515]:
]dinput
BFS←{              ⍝ nodes BFS node
    graph←⍺
    ⍬{
        0=≢⍵:⍺
        node←1↑⍵ ⋄ queue←1↓⍵
        ch←node⊃graph
        (⍺,node)∇(queue∪ch)~⍺
    }⍵
}

In [516]:
≢DAY12 BFS 0 ⍝ Part 1: 175

175

Part 2: find the number of connected components. We repeatedly apply the BFS routine above and mark visited nodes.

In [518]:
]dinput
ConnectedComponents←{
    graph←⍵
    seen←⍬
    0 {
        root←⍵
        root≥≢graph:⍺
        root∊seen:⍺∇⍵+1
        seen,←graph BFS root
        (⍺+1)∇⍵+1
    } 0
}

In [520]:
ConnectedComponents DAY12 ⍝ 213

213

### Day 13: Packet Scanners
https://adventofcode.com/2017/day/13

In [10]:
⎕IO←0
'segs'⎕CY'dfns'
DAY13←⍎¨¨' :'∘segs¨⊃⎕NGET'data/2017/13.txt'1 

In [11]:
MAX←⊃⊃1↑¯1↑DAY13
IDX←⊣/↑DAY13
LEN←⊢/↑DAY13

The `Pos` function returns the location of the scanner for a given picosecond and layer depth.

In [12]:
Pos←{cycle←⍺-1⋄1=2|⌊⍵÷cycle:cycle-cycle|⍵⋄cycle|⍵} ⍝ depth Pos pico - the index at pico, for given depth

For part 1 we traverse through once, keeping track of each layer where we got caught. The cost of getting caugfht is the layer index times its depth.

In [13]:
Day13p1←{fw←⍵⋄0{⍵≥≢fw:⍺⋄0=fw[⍵] Pos ⍵:(⍺+fw[⍵]×⍵)∇⍵+1⋄⍺∇⍵+1}0}

In [14]:
+/Day13p1 LEN@IDX⊢(1+MAX)⍴0 ⍝ Part 1: 2384

2384

In part 2, we need to find a delay that would enable us to traverse through all the layers without getting caught at all.

In [15]:
]dinput
Day13p2←{
    fw←⍵
    0 {                         ⍝ Current delay is ⍺
        ⍵≥≢fw:⍺
        0=fw[⍵]:⍺∇⍵+1           ⍝ Skip gaps
        0=fw[⍵] Pos ⍵+⍺:(⍺+1)∇0 ⍝ Caught! Increase delay and re-start from layer 0
        ⍺∇⍵+1                   ⍝ Still going: try the next layer
    } 0                         ⍝ Current layer
}

In [16]:
Day13p2 LEN@IDX⊢(1+MAX)⍴0  ⍝ Part 2: 3921270

3921270

### Day 14: Disk Defragmentation
https://adventofcode.com/2017/day/14

This makes use of the 'Knot' function, defined in Day 10.

In [39]:
⎕IO←0
'dec'⎕CY'dfns'   ⍝ Hex to dec helper function
DAY14←'jzgqcdpd'

In [41]:
+/∊{(8⍴2)⊤ dec Knot DAY14,'-',⍕⍵}¨⍳128  ⍝ Part 1: 8074

8074